In [6]:
#!pip install tensorflow

In [3]:
import tensorflow as tf 

In [5]:
print(tf.config.list_physical_devices('cpu'))

[]


In [7]:
import os
from pathlib import Path
import urllib.request as request
import numpy as np
import matplotlib.pyplot as plt

In [8]:
IMAGE_SIZE=(224,224)

In [9]:
model=tf.keras.applications.VGG16(
    #include_top=False->upto convolution layer
    include_top=True, weights="imagenet", input_tensor=None, input_shape=None,classes=1000
    
    
)

In [10]:
model.summary()

Model: "vgg16"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_1 (InputLayer)        [(None, 224, 224, 3)]     0         
                                                                 
 block1_conv1 (Conv2D)       (None, 224, 224, 64)      1792      
                                                                 
 block1_conv2 (Conv2D)       (None, 224, 224, 64)      36928     
                                                                 
 block1_pool (MaxPooling2D)  (None, 112, 112, 64)      0         
                                                                 
 block2_conv1 (Conv2D)       (None, 112, 112, 128)     73856     
                                                                 
 block2_conv2 (Conv2D)       (None, 112, 112, 128)     147584    
                                                                 
 block2_pool (MaxPooling2D)  (None, 56, 56, 128)       0     

In [11]:
model.save("VGG16_full_model.h5")

In [12]:
model.layers[1].padding

'same'

In [13]:
data_URL="https://download.pytorch.org/tutorial/hymenoptera_data.zip"

# Create a directory

In [14]:
def create_dirs(dir_path):
    os.makedirs(dir_path, exist_ok=True)
    print(f"{dir_path} directory created")

In [15]:
ROOT_DATA_DIR="data"
create_dirs(ROOT_DATA_DIR)

data directory created


# Downloading Data

In [16]:
data_zip_file="data.zip"
data_zip_file_path=os.path.join(ROOT_DATA_DIR,data_zip_file)


In [17]:
if not os.path.isfile(data_zip_file):
    print("Downloading dara...............................")
    filename, headers=request.urlretrieve(data_URL,data_zip_file_path)
    print(f"filename: {filename}  created with info \n{headers}")
else:
    print(f"file is already present")

filename: data\data.zip  created with info 
Content-Type: application/zip
Content-Length: 47286322
Connection: close
Date: Sat, 28 May 2022 14:19:56 GMT
Last-Modified: Wed, 15 Mar 2017 18:46:00 GMT
ETag: "5f8c32a6554f6acb4d649776e7735e48"
x-amz-version-id: null
Accept-Ranges: bytes
Server: AmazonS3
X-Cache: Miss from cloudfront
Via: 1.1 98b89ed6771805fd5b6d059738f20206.cloudfront.net (CloudFront)
X-Amz-Cf-Pop: TXL50-P3
X-Amz-Cf-Id: 2tF6V821dG3U3jIkzfmKdP1TiKLoBU0WPBMt1YRNpnpILOmmy2hRtg==


